In [ ]:

import pandas as pd
import random
import time

from LLM.gpt import send_requestGPT
from LLM.gemini import send_requestGemini
from LLM.llama import send_requestLlama

Check how many unique texts there are in the dataset

In [ ]:
def unique_values_MASTER():
    df = pd.read_csv("./data/preprocessed/MASTER.csv")
    unique_text_count = df['text'].nunique()
    print(f"Number of unique values in 'text' column: {unique_text_count}")

In [9]:
# Count remaining Nan values in MASTER
df = pd.read_csv("./data/result/MASTER.csv")
# print for each column ['gpt-4o-mini','gpt-4-0125-preview','gpt-3.5-turbo-0125','llama3.2-3b','llama3.1-8b','llama3-8b','gemini-1.5-flash',]]
print(f"Remaining Texts 'gpt-4o-mini' column: \t\t {df['gpt-4o-mini'].isnull().sum()} out of {len(df)}")
print(f"Remaining Texts 'gpt-4-0125-preview' column: \t {df['gpt-4-0125-preview'].isnull().sum()} out of {len(df)}")
print(f"Remaining Texts 'gpt-3.5-turbo-0125' column: \t {df['gpt-3.5-turbo-0125'].isnull().sum()} out of {len(df)}")
print(f"Remaining Texts 'llama3.2-3b' column: \t\t {df['llama3.2-3b'].isnull().sum()} out of {len(df)}")
print(f"Remaining Texts 'llama3.1-8b' column: \t\t {df['llama3.1-8b'].isnull().sum()} out of {len(df)}")
print(f"Remaining Texts 'llama3-8b' column: \t\t {df['llama3-8b'].isnull().sum()} out of {len(df)}")
print(f"Remaining Texts 'gemini-1.5-flash' column: \t {df['gemini-1.5-flash'].isnull().sum()} out of {len(df)}")


Remaining Texts 'gpt-4o-mini' column: 		 0 out of 0
Remaining Texts 'gpt-4-0125-preview' column: 	 0 out of 0
Remaining Texts 'gpt-3.5-turbo-0125' column: 	 0 out of 0
Remaining Texts 'llama3.2-3b' column: 		 0 out of 0
Remaining Texts 'llama3.1-8b' column: 		 0 out of 0
Remaining Texts 'llama3-8b' column: 		 0 out of 0
Remaining Texts 'gemini-1.5-flash' column: 	 0 out of 0


In [ ]:
def populate_row(model):
    
    df = pd.read_csv("./data/result/MASTER.csv")
    system_prompt = "You are an assistant trained to identify if text contains sexism. Answer ONLY with '1' for Yes or '0' for No. ONLY CHECK FOR SEXISM AND NOT OTHER FORMS OF HATE SPEECH."
    
    nan_rows = df[df[model].isna()]

    if not nan_rows.empty:
        random_index = random.choice(nan_rows.index)
        text_value = df.at[random_index, 'text']
        sexism = df.at[random_index, 'SEXISM']
        if "gemini" in model:
            value = send_requestGemini(model, system_prompt, text_value)
        elif "llama" in model:
            value = send_requestLlama(model, system_prompt, text_value)            
        else:
            value = send_requestGPT(model, system_prompt, text_value)
        df.at[random_index, model] = value
        print(f"Populated NaN at index {random_index} with value {value} vs. {sexism} for text: {text_value}") 
    else:
        print(f"No NaN values found in {model} column.")

    df.to_csv("./data/result/MASTER.csv", index=False)


In [ ]:
# model options: "gpt-4o-mini", "gpt-4-0125-preview", "gpt-3.5-turbo-0125"
for i in range(1000):
    populate_row("gpt-4o-mini")

In [8]:
# LLAMA 
for i in range(1000):
    populate_row("llama3.2-3b")

Populated NaN at index 395 with value 1 vs. 0 for text: It's so sick and sad that young people will think of the concept of clout like rap niggers invented it because they aren't even properly taught about the english language anymore.
Populated NaN at index 9381 with value 0 vs. 0 for text: Good thinking law enforcement the spic UC in the Honda Accord can eat my shit say one word mind games are fun be th\u2026 @URL
Populated NaN at index 4908 with value 0 vs. 0 for text: new immigrants one warned us halloween hadnt quite gone global yet early 80s. reme @url
Populated NaN at index 5594 with value 0 vs. 0 for text: @user learn spell mongy
Populated NaN at index 7215 with value 0 vs. 0 for text: @user @user @user mongol will always have a spot in any of my teams
Populated NaN at index 8942 with value 1 vs. 0 for text: Cuz guess what faggot I made the blue print
Populated NaN at index 2822 with value 0 vs. 0 for text: nigger put paint minute maid bottle @url
Populated NaN at index 5997 wi

KeyboardInterrupt: 

In [ ]:
# GEMINI
for i in range(1000):
    try:
        populate_row("gemini-1.5-flash")
    except:
        print("Error occured")
    time.sleep(4) # to avoid rate limiting

In [ ]:
import pandas as pd
import random
import time

from LLM.gpt import send_requestGPT
from LLM.gemini import send_requestGemini

from openai import OpenAI

client = OpenAI()
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I love you"
)
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id="asst_fADAB9avOLE36JcUYQWRjece"
)


In [ ]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
else:
  print(run.status)

In [ ]:
df = pd.read_csv("./data/result/MASTER.csv")
# reorder columns to the following: ID,dataset,SEXISM,gpt-4o-mini,gpt-4-0125-preview,gpt-3.5-turbo-0125,llama3.2-3b,llama3.1-8b,llama3-8b,gemini-1.5-flash,text
#df['llama3.2-3b'] = None
#df['llama3.1-8b'] = None
#df['llama3-8b'] = None
df = df[['ID','dataset','SEXISM','gpt-4o-mini','gpt-4-0125-preview','gpt-3.5-turbo-0125','llama3.2-3b','llama3.1-8b','llama3-8b','gemini-1.5-flash','text']]
df.to_csv("./data/result/MASTER.csv", index=False)